# Example 002 - estimate SIM performance using Argolight SIM v1 test target slide
  
Test targets with known ground truth and a variety of pattern orientations are useful to estimate the overall quality of the SIM instrument and reconstruction. Here, we use an [Argolight SIM v1 slide](https://argolight.com/files/Argo-SIM/Argo-SIM-v1.1_User-guide.pdf) with sets of lines that are spaced from 0 to 390 nm apart. The line spacing changes in 30 nm increments.

### Import libraries.

In [ ]:
import napari
import numpy as np
from numpy import fft
import pickle
import tifffile
from pathlib import Path
import mcsim.analysis.sim_reconstruction as sim
import localize_psf.fit_psf as psf
import localize_psf.affine as affine
import localize_psf.rois as rois
from mcsim.analysis import analysis_tools
import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm

### Load experimental data containing all wavelengths and patterns.

In [ ]:
ncolors = 2
nangles = 3
nphases = 3
nx = 2048
ny = 2048
imgs = tifffile.imread(Path("data", "example_002", "raw_data", "argosim_line_pairs.tif")).reshape([ncolors, nangles, nphases, ny, nx])

### Set experimental metadata.

In [ ]:
na = 1.3
pixel_size = 0.065
excitation_wavelengths = [0.465, 0.532]
emission_wavelengths = [0.519, 0.580]

### View data in Napari

In [ ]:
viewer = napari.view_image(imgs[0,:],name='em: 520 nm',scale=(pixel_size,pixel_size))
viewer.add_image(imgs[1,:],name='em: 580 nm',scale=(pixel_size,pixel_size))
viewer.scale_bar.unit='um'
viewer.scale_bar.visible=True

### Crop image to ROI containing Argolight test target.

In [ ]:
roi = rois.get_centered_roi([791, 896], [850, 850])
nx_roi = roi[3] - roi[2]
ny_roi = roi[1] - roi[0]

## SIM reconstruction using calibrated instrument parameters 

### Load experimental optical transfer function.

In [ ]:
otf_data_path = Path("data", "example_002", "calibration", "2020_05_19_otf_fit_blue.pkl")

with open(otf_data_path, 'rb') as f:
    otf_data = pickle.load(f)
otf_p = otf_data['fit_params']

otf_fn = lambda f, fmax: 1 / (1 + (f / fmax * otf_p[0]) ** 2) * psf.circ_aperture_otf(f, 0, na, 2 * na / fmax)

### Load pre-calculated affine transforms for qi2lab DMD-SIM.

In [ ]:
affine_fnames = [Path("data", "example_002", "calibration", "2021-02-03_09;43;06_affine_xform_blue_z=0.pkl"),
                 Path("data", "example_002", "calibration", "2021-02-03_09;43;06_affine_xform_green_z=0.pkl")]

affine_xforms = []
for p in affine_fnames:
    with open(p, 'rb') as f:
        affine_xforms.append(pickle.load(f)['affine_xform'])

### Load qi2lab DMD-SIM patterns. Estimate frequency and phase using extracted affine transforms.

In [ ]:
dmd_pattern_data_fpath = [Path("data", "example_002", "calibration", "sim_patterns_period=6.01_nangles=3.pkl"),
                          Path("data", "example_002", "calibration", "sim_patterns_period=6.82_nangles=3.pkl")]

frqs_dmd = np.zeros((2, 3, 2))
phases_dmd = np.zeros((ncolors, nangles, nphases))
for kk in range(ncolors):
    ppath = dmd_pattern_data_fpath[kk]
    xform = affine_xforms[kk]

    with open(ppath, 'rb') as f:
        pattern_data = pickle.load(f)

    # DMD intensity frequency and phase (twice electric field frq/phase)
    frqs_dmd[kk] = 2 * pattern_data['frqs']
    phases_dmd[kk] = 2 * pattern_data['phases']
    dmd_nx = pattern_data['nx']
    dmd_ny = pattern_data['ny']

for kk in range(1):
    # otf matrix
    fmax = 1 / (0.5 * emission_wavelengths[kk] / na)
    fx = fft.fftshift(fft.fftfreq(nx_roi, pixel_size))
    fy = fft.fftshift(fft.fftfreq(ny_roi, pixel_size))
    ff = np.sqrt(fx[None, :] ** 2 + fy[:, None] ** 2)
    otf = otf_fn(ff, fmax)
    otf[ff >= fmax] = 0

    # guess frequencies/phases
    frqs_guess = np.zeros((nangles, 2))
    phases_guess = np.zeros((nangles, nphases))
    for ii in range(nangles):
        for jj in range(nphases):
            # estimate frequencies based on affine_xform
            frqs_guess[ii, 0], frqs_guess[ii, 1], phases_guess[ii, jj] = \
                affine.xform_sinusoid_params_roi(frqs_dmd[kk, ii, 0], frqs_dmd[kk, ii, 1],
                                                 phases_dmd[kk, ii, jj], [dmd_ny, dmd_nx], roi, xform)

    # convert from 1/mirrors to 1/um
    frqs_guess = frqs_guess / pixel_size

### Perform SIM reconstruction using known OTF and plot results.

In [ ]:
%matplotlib inline
imgset = sim.SimImageSet({"pixel_size": pixel_size, "na": na, "wavelength": emission_wavelengths[0]},
                        imgs[0, :, :, roi[0]:roi[1], roi[2]:roi[3]],
                        frq_estimation_mode="band-correlation",
                        frq_guess=frqs_guess,
                        phases_guess=phases_guess,
                        phase_estimation_mode="wicker-iterative",
                        combine_bands_mode="fairSIM",
                        fmax_exclude_band0=0.4, 
                        normalize_histograms=True,
                        otf=otf,
                        wiener_parameter=0.1,
                        background=100, 
                        gain=2, 
                        min_p2nr=0.5,
                        save_dir=Path("data","example_002","reconstruction"),
                        interactive_plotting=False, 
                        figsize=(20, 13))

imgset.reconstruct()
imgset.plot_figs()
imgset.save_imgs()
imgset.log_file.close()

wf_images = imgset.widefield
SR_images = imgset.sim_sr

del imgset

### Display results

In [ ]:
colormaps = ['bop blue', 'bop orange']

viewer = napari.view_image(wf_images,name='Widefield',scale=(pixel_size,pixel_size),colormap = colormaps[0],blending='additive')
viewer.add_image(SR_images,name='SR',scale=(pixel_size/2,pixel_size/2),colormap = colormaps[1],blending='additive')

viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

## Importance of SLM calibration

### Calculate FFT of raw data to find frequency guesses

In [ ]:
%matplotlib widget
for ch_idx in range(ncolors):

    image_set = imgs[ch_idx,:,:,:,:]

    dx = pixel_size
    fxs = analysis_tools.get_fft_frqs(nx, dx)
    df = fxs[1] - fxs[0]
    fys = analysis_tools.get_fft_frqs(ny, dx)
    ff = np.sqrt(np.expand_dims(fxs, axis=0)**2 + np.expand_dims(fys, axis=1)**2)
    for ii in range(image_set.shape[0]):
        ft = fft.fftshift(fft.fft2(fft.ifftshift(np.squeeze(image_set[ii, 0, :]))))

        figh = plt.figure()
        plt.title('Channel='+str(ch_idx)+", Angle="+str(ii))
        plt.imshow(np.abs(ft), norm=PowerNorm(gamma=0.1,vmin=65,vmax=1.5e7),
                    extent=[fxs[0] - 0.5 * df, fxs[-1] + 0.5 * df, fys[-1] + 0.5 * df, fys[0] - 0.5 * df])
        plt.show()